## Great, now that we discussed a little let's continue

Given that the current approach utilized by the authors lacks reproducibility, we will explore an alternative method by leveraging nf-core pipelines for data analysis.

Please explain, how we will achieve reproducibility for the course  with this approach.


Because the nf-core pipelines are packaged software, we should receive the same results when re-running the pipeline, regardless of OS or user, since all software versions will be the same (as specified by the Container).

You have successfully downloaded 2 of the fastq files we will use in our study.

What is the next step if we want to first have a count table and check the quality of our fastq files? What is the pipeline called to do so?

Since this is RNA-seq data we can use nf-core/rnaseq pipeline to check the quality and produce the count table. Therefore, the next step is to run the rnaseq pipeline using a samplesheet and our FASTQ files as input.

Analyze the 2 files using an nf-core pipeline.

What does this pipeline do?
It performs quality control, trimming and (pseudo-)alignment to quantify gene expression levels, thereby producing a gene expression matrix and QC report.

Which are the main tools that will be used in the pipeline?
FastQC for quality control, STAR/HiSAT2 for multiple alignment, Salmon/RSEM for quantification (Salmon and Kallisto can also be used for pseudoalignment), SAMtools for sorting and indexing the reads, and Picard to mark the duplicates.

As all other nf-core pipelines, the chosen pipeline takes in a samplesheet as input.

Use Python and pandas to create the samplesheet for your 2 samples. Feel free to make use of the table you created earlier today.

Choose your sample names wisely, they must be the connection of the results to the metadata. If you can't find the sample in the metadata later, the analysis was useless.

In [7]:
import pandas as pd

data = {
    'sample': ['SHAM_OXY', 'SNI_OXY'],
    'fastq_1': ['files/fastq/SRX19144488_SRR23195511_1.fastq.gz', 'files/fastq/SRX19144486_SRR23195516_1.fastq.gz'],
    'fastq_2': ['files/fastq/SRX19144488_SRR23195511_2.fastq.gz', 'files/fastq/SRX19144486_SRR23195516_2.fastq.gz'],
    'strandedness': ['auto', 'auto']
}

# Create DataFrame
df = pd.DataFrame(data)

# Write to CSV
df.to_csv('samplesheet.csv', index=False)

In [15]:
# post here the command you used to run nf-core/rnaseq

!nextflow run nf-core/rnaseq --input samplesheet.csv --outdir output --gtf genomic.gtf --fasta GCF_000001635.27_GRCm39_genomic.fna -profile docker --max_memory=8GB --max_cpus=8


 N E X T F L O W   ~  version 24.04.4

NOTE: Your local project version looks outdated - a different revision is available in the remote repository [1f3f64dac7]
Launching `https://github.com/nf-core/rnaseq` [angry_kalam] DSL2 - revision: 33df0c05ef [master]

WARN: Access to undefined parameter `monochromeLogs` -- Initialise it to a default value eg. `params.monochromeLogs = some_value`


------------------------------------------------------
                                        ,--./,-.
        ___     __   __   __   ___     /,-._.--~'
  |\ | |__  __ /  ` /  \ |__) |__         }  {
  | \| |       \__, \__/ |  \ |___     \`-._,-`-,
                                        `._,._,'
  nf-core/rnaseq v3.16.0-g33df0c0
------------------------------------------------------
Core Nextflow options
  revision        : master
  runName         : angry_kalam
  containerEngine : docker
  launchDir       : /Users/Jessie/PycharmProjects/comp_workflows/day2
  workDir         : /Users/Jessie/Pycharm

Explain all the parameters you set and why you set them in this way.

--input is the samplesheet containing the filepath of the fastq sequencing files to be used, as well as the names of the samples.
--outdir specifies the output directory where the output files should be saved
--gtf specifies the .gtf file containing genome annotations of the reference genome
--fasta specifies the reference genome sequence
-profile is used to specify docker
--max_memory specifies maximum allowed memory usage
--max_cpus specifies maximum allowed CPU usage

## Browsing the results

How did the pipeline perform?

It required a lot of memory and cpu, resulting in a very long runtime, but it produced the expected results (results received from tutors).

Explain the quality control steps. Are you happy with the quality and why. If not, why not. <br>
Quality control consists of multiple steps. FastQC is run on the raw sequencing reads to determine the quality scores for each sample/read (as Phred scores). Additional checks include: <br>
1. determining the GC content (does it match what is expected)
2. length distribution
3. duplication levels (can indicate PCR over-amplification)

The MultiQC report aggregates the results from multiple quality chcks across different samples.
Overall, the quality looks good and the reads show high Phred quality scores, indicating that the sequencing was successful. Two samples did, however, fail the strandedness check.


Please give additional information on : 
- ribosomal rRNA
Ideally, we would like a very low level (if any) of rRNA contamination, since we are interested in sequencing mRNA and not rRNA, in order to investigate gene expression levels. SNI_Sal_2 and SNI_Sal_4 have relatively high %rRNA values of 22.08% and 25.10%, respectively, which is not ideal.

- Duplication
The trimmed sequence duplication levels indicate that 19 out of the 32 samples failed due to high duplication levels which could indicate some kind of enrichment bias (e.g. PCR over amplification). Spikes towards the right end of the FastQC duplication plot could indicate specific enrichments of subsets. SNI_Sal_2 and SNI_Sal_4 showed this spike (and had the highest level of duplication overall), which could mean that the high duplication level is due to their specific enrichment of rRNA.

- GC content
If we look at the per sequence GC content of the trimmed sequences (FastQC), we see that 14 out of 32 samples failed the quality check because the %GC has two peaks, which could indicate contamination. 

What are the possible steps that could lead to poorer results?

The high level of rRNA contamination could indicate that library prep (rRNA removal) was not performed properly.

Would you exclude any samples? If yes, which and why?

I would think of excluding SNI_Sal_2 and SNI_Sal_4 due to their rRNA contamination and high levels of duplication. These samples also had a low percentage of properly paired reads (both reads in the pair are mapped and fall within an acceptable range from each other). These 2 samples also failed the strandedness check between that provided in the samplesheet and calculated by the RSeQC infer_experiment.py tool, and were also identified as outliers in the DESeq2 PCA plot.


Sham_oxy_1 had a lot of unmapped reads because they were too short, which could also be a reason to exclude this sample.

What would you now do to continue the experiment? What are the scientists trying to figure out? Which packages on R or python would you use?

To continue, we should look at and compare the gene expression levels between samples. We can run the pipeline with the tool Salmon to quantify gene expression levels and obtain a gene count table (matrix) to compare gene counts and transcript abundance between samples. This will indicate if there are any significant differences in gene expression between samples, and from there we can start to draw conclusions.